# TensorboardX

- logging scalar, image, audio, histogram, text, embedding, route of back-propagation을 지원함

## Install

```bash
> pip3 install tensorboardX
```

## Execute

```bash
> tensorboard --logdir=<your log dir>
```

### Tip  
> `~/.bashrc`에 `tb='tensorboard --logdir'`을 적용하면 다음과 같이 사용할 수 있다.  
> `tb <your log dir>`

# Create a summary writer

- logging하기 이전에, writer instance를 생성해줘야함

In [ ]:
from tensorboardX import SummaryWriter

# SummaryWriter encapsulates everything
# creates writer object. The Log will be saved in 'runs/exp-1'
writer = SummaryWriter('runs/exp-1')

# creates writer2 object with auto generated file name, 
# the dir will be something like 'runs/Aug20-17-20-33'
writer2 = SummaryWriter()

#creates writer3 object with auto generated file name, 
# the comment will be appended to the filename. 
# The dir will be something like 'runs/Aug20-17-20-33-3xlearning rate'
writer3 = SummaryWriter(comment='3x learning rate')

# General api format

```python
add_something(tage name, object, iteration number)
```

## Add scalar
```python
writer.add_scalar('myscalar', value, iteration)
```

- 여기서 value 파라미터에서 만약 `x`가 `torch scalar tensor`라면, `x.item()`으로 넣어줘야한다.

## Add image
```python
writer.add_image('imresult', x, iteration)
```

- image는 [3, H, W] 형태로 3차원 텐서다.
- 만약에 image가 batch라면 `torchvision`의 `make_grid`를 사용해서 나온 결과를 입력으로 넣어야한다.  
> `make_grid`는 4D Tensor를 3D Tensor로 변환해준다.  
> Image는 normalize해야한다.

## Add histogram
```python
writer.add_histogram('hist', array, iteration)
```

- histogram을 저장하는 것은 computation time 및 저장 비용이 크다.
- histogram을 저장하려면, `torch.tensor`를 `numpy.array`로 변경해줘야한다.
> 학습 후, 속도가 느려진다면, histogram 저장하는 부분부터 확인하는 것이 좋다.

## Add figure
- `matplotlib` figure를 tensorboard에 저장하고 싶을 때, `add_figure`함수를 사용하면 된다.
- **figure**는 `matplotlib.pyplot.figure`거나, `matplotlib.pyplot.figure`로 구성된 `list`여야 한다.  
([api 문서](https://tensorboardx.readthedocs.io/en/latest/tensorboard.html#tensorboardX.SummaryWriter.add_figure)를 참조하자)  

## Add graph
- 모델을 visualization하려면, model과 input이 있어야한다.
- 여기서 input은 model에 의존하는 tensor의 list이다.
- `model(input)`이 작동하는지 확인하고, 사용해야한다.
([The graph demo](https://github.com/lanpa/tensorboardX/blob/master/examples/demo_graph.py)를 확인하자)

## Add audio
- 현재 sampling rate이 **44100 kHz**로, 그리고 Single channel로 고정되어있다.
- `add_audio`함수의 입력은 1D array이다.

## Add embedding
- Embedding은 고차원 데이터.
- 사람이 볼 수 있는 3D 데이터로 변환해서 보여준다. 이때, PCA나 `t-sne`를 사용해서 저차원으로 project한다.
- Input은 데이터 array만 전달하면 되고, 나머지는 Tensorboard가 알아서 다 해준다.
- 이때, Input array는 `n x d`가 된다. `n`은 데이터 포인트이고, `d`는 데이터 차원이다.  

<br/>

- feature representation은 raw data이거나, network가 학습한 representation일 수 있다.
- 이러한 것들은 data point들의 분포를 결정한다.
- 시각화를 하는데, 조금 더 직관적으로 보여주기 위해서 **metadata**나 각 data point에 대한 **label_img**들을 인자로 줄 수 있다.
- **metadata**는 label들의 list이고, 리스트의 길이는 point의 수인 `n`과 같아야한다.
- **label_img**들은 [N, C, H, W] 형태를 갖는 4D Tensor이고, N은 `n`과 같아야한다.  
([The embedding demo](https://github.com/lanpa/tensorboardX/blob/master/examples/demo_embedding.py)를 확인하자)

# Demo

In [3]:
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter

resnet18 = models.resnet18(False)
writer = SummaryWriter("./demo")
sample_rate = 44100
freqs = [262, 294, 330, 349, 392, 440, 440, 440, 440, 440, 440]

for n_iter in range(100):

    dummy_s1 = torch.rand(1)
    dummy_s2 = torch.rand(1)
    # data grouping by `slash`
    writer.add_scalar('data/scalar1', dummy_s1[0], n_iter)
    writer.add_scalar('data/scalar2', dummy_s2[0], n_iter)

    writer.add_scalars('data/scalar_group', {'xsinx': n_iter * np.sin(n_iter),
                                             'xcosx': n_iter * np.cos(n_iter),
                                             'arctanx': np.arctan(n_iter)}, n_iter)

    dummy_img = torch.rand(32, 3, 64, 64)  # output from network
    if n_iter % 10 == 0:
        x = vutils.make_grid(dummy_img, normalize=True, scale_each=True)
        writer.add_image('Image', x, n_iter)

        dummy_audio = torch.zeros(sample_rate * 2)
        for i in range(x.size(0)):
            # amplitude of sound should in [-1, 1]
            dummy_audio[i] = np.cos(freqs[n_iter // 10] * np.pi * float(i) / float(sample_rate))
        writer.add_audio('myAudio', dummy_audio, n_iter, sample_rate=sample_rate)

        writer.add_text('Text', 'text logged at step:' + str(n_iter), n_iter)

        for name, param in resnet18.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), n_iter)

        # needs tensorboard 0.4RC or later
        writer.add_pr_curve('xoxo', np.random.randint(2, size=100), np.random.rand(100), n_iter)

dataset = datasets.MNIST('mnist', train=False, download=True)
images = dataset.test_data[:100].float()
label = dataset.test_labels[:100]

features = images.view(100, 784)
writer.add_embedding(features, metadata=label, label_img=images.unsqueeze(1))

# export scalar data to JSON for external processing
writer.export_scalars_to_json("./all_scalars.json")
writer.close()

In [6]:
!tensorboard --logdir="./demo"

Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 7, in <module>
    from tensorboard.main import main
ImportError: cannot import name 'main'
